# Using the ModelMW python client to run GWLF-E model in bulk

## Import libraries

In [1]:
from IPython.display import display

from modelmw_client import *

## Create an API client

In this case, I'm importing my saved credential information from a separate python file, but the credentials can be stored anywhere.

In [2]:
from mmw_secrets import (
    srgd_staging_api_key,
    srgd_mmw_user,
    srgd_mmw_pass,
)

# Create an API user
mmw_run = ModelMyWatershedAPI(api_key=srgd_staging_api_key, use_staging=True)
# Authenticate with MMW, optional
mmw_run.login(mmw_user=srgd_mmw_user, mmw_pass=srgd_mmw_pass)


True

## Set up the areas of interest

To run the model on a batch of areas of interest, we need a list of those areas.
We can either have a list of HUC codes (8, 10, or 12 digit HUCs) or a list of single-feature geojsons.
The list can be imported from a csv or other file.

If you're using geojsons, I recommend using the [python geojson library](https://pypi.org/project/geojson/) in combination with [GeoPandas](https://geopandas.org/en/stable/) or [SpatialPandas](https://github.com/holoviz/spatialpandas).
All of those packages use [fiona](https://fiona.readthedocs.io/en/latest/manual.html) for reading and writing geometry files.

For this example, we're just going to use a simple list of HUC codes.
These are the HUC 12's within the White Clay Creek HUC 10.

In [3]:
hucs_to_analyze = [
    "020402050301",
    "020402050302",
    "020402050303",
    "020402050304",
    "020402050305",
    "020402050306",
    "020402050307",
    "020402050308",
]


## Run model preparation (MapShed) and then GWLF-E

Using the ModelMyWatershed client, running GWLF-E in bulk is a single line of code.
By default, ModelMyWatershed will use the 2019 NLCD land use data for the analysis.
To change the base layers, you can submit a dictionary of layer overrides.

Underneath, the ModelMyWatershed client is making multiple requests to prepare data and get results.
First, it starts a preparation job to collect all of the land use, soil, geology, and other data about the area of interst needed by the model.
Historically, this prepartion was done using a Windows-based program called "MapShed" and that name remains in many places in ModelMyWatershed.
ModelMyWatershed returns a job id for the preparation job rather than to processed results.
The client then asks ModelMyWatershed to start another job to run GWLF-E itself.
The input for GWLF-E can either be a "z" file (another MapShed carry-over) with the preparation outputs, or the UUID of the preparation job.
Once the GWLF-E job is finished, the client fetches the results from the job endpoint.


In [4]:
gwlfe_result_dict = mmw_run.run_batch_gwlfe(hucs_to_analyze)

for name, frame in gwlfe_result_dict.items():
    print(name)
    display(frame)
    print("***")

	gwlf-e_prepare job started for 020402050301
	Got gwlf-e_prepare results for 020402050301
	gwlf-e_run job started for 020402050301
	Got gwlf-e_run results for 020402050301
	gwlf-e_prepare job started for 020402050302
	Got gwlf-e_prepare results for 020402050302
	gwlf-e_run job started for 020402050302
	Got gwlf-e_run results for 020402050302
	gwlf-e_prepare job started for 020402050303
	Got gwlf-e_prepare results for 020402050303
	gwlf-e_run job started for 020402050303
	Got gwlf-e_run results for 020402050303
	gwlf-e_prepare job started for 020402050304
	Got gwlf-e_prepare results for 020402050304
	gwlf-e_run job started for 020402050304
	Got gwlf-e_run results for 020402050304
	gwlf-e_prepare job started for 020402050305
	Got gwlf-e_prepare results for 020402050305
	gwlf-e_run job started for 020402050305
	Got gwlf-e_run results for 020402050305
	gwlf-e_prepare job started for 020402050306
	Got gwlf-e_prepare results for 020402050306
	gwlf-e_run job started for 020402050306
	Got gwlf

,AvPrecipitation,AvEvapoTrans,AvGroundWater,AvRunoff,AvStreamFlow,AvPtSrcFlow,AvTileDrain,AvWithdrawal,month,job_label
0,7.924800,0.560595,3.551375,0.832671,4.384046,0.0,0.0,0.0,1,020402050301
1,7.248737,0.853556,4.426393,0.797996,5.224389,0.0,0.0,0.0,2,020402050301
2,8.751993,2.475940,5.693435,0.688869,6.382303,0.0,0.0,0.0,3,020402050301
3,8.906510,5.248064,5.631621,0.120756,5.752377,0.0,0.0,0.0,4,020402050301
4,9.634643,9.775066,4.232908,0.175309,4.408217,0.0,0.0,0.0,5,020402050301
...,...,...,...,...,...,...,...,...,...,...
91,9.152890,10.321928,0.417342,0.522405,0.939747,0.0,0.0,0.0,8,020402050308
92,8.694420,6.123376,0.178185,0.605637,0.783822,0.0,0.0,0.0,9,020402050308
93,6.980767,4.217954,0.284595,0.655966,0.940560,0.0,0.0,0.0,10,020402050308
94,8.397240,2.373579,0.363544,1.091715,1.455259,0.0,0.0,0.0,11,020402050308


***
gwlfe_load_summaries


,Source,Unit,Sediment,TotalN,TotalP,job_label
0,Total Loads,kg,2.249408e+06,63046.752599,3563.654890,020402050301
1,Loading Rates,kg/ha,6.372081e+02,17.859764,1.009505,020402050301
2,Mean Annual Concentration,mg/l,1.733649e+02,4.859096,0.274656,020402050301
3,Mean Low-Flow Concentration,mg/l,1.039892e+03,7.150891,1.360312,020402050301
4,Total Loads,kg,1.306242e+06,44695.541619,2186.430141,020402050302
5,Loading Rates,kg/ha,5.075345e+02,17.366259,0.849528,020402050302
6,Mean Annual Concentration,mg/l,1.475574e+02,5.048957,0.246986,020402050302
7,Mean Low-Flow Concentration,mg/l,7.694098e+02,6.516539,1.302115,020402050302
8,Total Loads,kg,5.348609e+06,129262.340449,7030.727169,020402050303
9,Loading Rates,kg/ha,6.168459e+02,14.907604,0.810842,020402050303


***
gwlfe_lu_loads


...

***
gwlfe_metadata


,NYrs,NRur,NUrb,NLU,SedDelivRatio,WxYrBeg,WxYrEnd,job_label
0,30,10,6,16,0.154784,1961,1990,020402050301
1,30,10,6,16,0.165277,1961,1990,020402050302
2,30,10,6,16,0.119215,1961,1990,020402050303
3,30,10,6,16,0.144688,1961,1990,020402050304
4,30,10,6,16,0.164874,1961,1990,020402050305
5,30,10,6,16,0.135479,1961,1990,020402050306
6,30,10,6,16,0.127343,1961,1990,020402050307
7,30,10,6,16,0.126362,1961,1990,020402050308


***
gwlfe_summaries


,AreaTotal,MeanFlow,MeanFlowPerSecond,inputmod_hash,watershed_id,job_label
0,3530.1,1.297500e+07,0.411434,,None,020402050301
1,2573.7,8.852431e+06,0.280709,,None,020402050302
2,8670.9,3.309287e+07,1.049368,,None,020402050303
3,4534.4,1.859122e+07,0.589524,,None,020402050304
4,2603.7,1.189725e+07,0.377259,,None,020402050305
5,5654.8,2.148129e+07,0.681167,,None,020402050306
6,6886.4,2.399005e+07,0.760720,,None,020402050307
7,7130.3,2.475211e+07,0.784884,,None,020402050308


***


## Finished!

Mind-boggling, isn't it?